In [5]:
### 方法参考来之：https://www.cnblogs.com/always-fight/p/10159547.html
## 2019-11-3

In [6]:
import sys
sys.version

'3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]'

In [1]:
import numpy as np
import pandas as pd
import re
import jieba

In [2]:
df_technology = pd.read_csv("./NLPData/technology_news.csv",encoding ="utf-8")

In [3]:
df_technology.head()

,Unnamed: 0,content
0,0,",中新网,1月7日电 恰逢CES 2017拉开大幕，却惊闻“AlphaGo升级版”的M..."
1,1,徐立，商汤科技CEO在谈起本次参展时谈到：“作为一个成立刚刚两年的创业公司，这次参展，一...
2,2,正如最后挑战Master的古力在落败后发表的观点：“人类与人工智能共同探索围棋世界的大幕...
3,3,", SenseFace人脸布控的“黑科技”"
4,4,本届大展最大的看点，无疑是“被誉为2016全美科技第一神股”英伟达的首次CES主题演讲。...


### 1.加载数据

In [6]:
import pandas as pd
# df_technology
df_technology  = pd.read_csv("./NLPData/technology_news.csv", encoding = 'utf-8')
df_technology  = df_technology.dropna() # 取消可能为空的行数
# sprots
df_sprots   = pd.read_csv("./NLPData/sports_news.csv", encoding = 'utf-8')
df_sprots   = df_sprots.dropna()

#df_society
df_society = pd.read_csv("./NLPData/society_news.csv",encoding = 'utf-8')
df_society = df_society.dropna()

#military
df_military   = pd.read_csv("./NLPData/military_news.csv", encoding = 'utf-8')
df_military   = df_military.dropna()

df_internality = pd.read_csv("./NLPData/international_news.csv",encoding = 'utf-8')
df_international = df_internality.dropna()

df_house = pd.read_csv("./NLPData/house_news.csv",encoding = 'utf-8')
df_house = df_house.dropna()

df_home = pd.read_csv("./NLPData/home_news.csv",encoding = 'utf-8')
df_home = df_home.dropna()

df_finance = pd.read_csv("./NLPData/finance_news.csv",encoding = 'utf-8')
df_finance = df_finance.dropna()

# df_entertainment 
df_entertainment   = pd.read_csv("./NLPData/entertainment_news.csv", encoding = 'utf-8')
df_entertainment   = df_entertainment.dropna()

# df_car
df_car  = pd.read_csv("./NLPData/car_news.csv", encoding = 'utf-8')
df_car  = df_car.dropna()

In [7]:
print (df_car.content.values.tolist()[11778])

　　神州专车高考生免费送考公益活动已持续三年，今年将覆盖30余城市，预计服务数千名考生。神州专车自2015年初在全国60大城市上线，以“专业司机、专业车辆”模式，为用户提供安全、舒适、标准化的出行服务。根据罗兰贝格报告，神州专车占据40%的专车市场份额，消费者满意度和安全评分均位居行业首位。


In [8]:
### stopwords ###
stopword_list = [k.strip() for k in open("./data/stopwords_NLP.txt",encoding ='utf-8') if k.strip()!= '']

### 1.1下面的代码用来采集全部的样本数据

In [13]:
# 读取文件里面的全部的csv文件 ###
import os
import csv
import pandas as pd
import time

cutWords = []
example  = 0
t1 = time.time()
for root,dirs,files in os.walk("./NLPData"):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            label    = os.path.splitext(file)[0] # 以文件的名字做为标签区分 #
            doc      = pd.read_csv(os.path.join(root,file),encoding = 'utf-8')
            contents = doc.content.dropna()
            for each_line in contents:
                if len(each_line)>20:
                    example +=1
                    cutWord = [word for word in jieba.lcut(each_line) if (word not in stopword_list and (word.strip()!= ''or word.lstrip()!=''))]
                    cutWords.append([" ".join(cutWord),label])
                    if example%10000 == 0:
                        t2 = time.time()
                        print("处理完前%d个样本所花费的时间是%0.2f"%(example,(t2 - t1)/60),'mins')

### 1.2采集超过特定长度，数量相同的个分类样本

In [56]:
import time
t1 = time.time()
def read_files(file,cutWords,label,limit,maxs,i=0):
    doc     = pd.read_csv(file,encoding = 'utf-8')
    content = doc.content.dropna()
    for each_line in content:
        if len(each_line)>limit:
            i = i+1
            if i<=maxs:
                cutWord = [word for word in jieba.lcut(each_line) if (word not in stopword_list and (word.strip()!= ''or word.lstrip()!=''))]
                cutWords.append([" ".join(cutWord),label])
            if i%20000 == 0:
                print("处理完前%d个样本所花费的时间是%0.2f"%(i,(time.time()- t1)/60),'mins')

In [58]:
#calling 
limit = 50   #采集的样本长度大于这个值
maxs  = 8000  # 每个总类的样本数量都是maxs 
cutWords = []
for root,dirs,files in os.walk("./NLPData"):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            label    = os.path.splitext(file)[0] # 以文件的名字做为标签区分 #
            file     = os.path.join(root,file)
            read_files(file,cutWords,label,limit,maxs)

处理完前10000个样本所花费的时间是0.97 mins
处理完前20000个样本所花费的时间是0.97 mins
处理完前10000个样本所花费的时间是1.32 mins
处理完前20000个样本所花费的时间是1.32 mins
处理完前30000个样本所花费的时间是1.32 mins
处理完前40000个样本所花费的时间是1.32 mins
处理完前50000个样本所花费的时间是1.32 mins
处理完前60000个样本所花费的时间是1.32 mins
处理完前70000个样本所花费的时间是1.32 mins
处理完前80000个样本所花费的时间是1.32 mins
处理完前90000个样本所花费的时间是1.32 mins
处理完前100000个样本所花费的时间是1.32 mins
处理完前110000个样本所花费的时间是1.32 mins
处理完前10000个样本所花费的时间是1.67 mins
处理完前20000个样本所花费的时间是1.67 mins
处理完前30000个样本所花费的时间是1.67 mins
处理完前40000个样本所花费的时间是1.67 mins
处理完前50000个样本所花费的时间是1.67 mins
处理完前60000个样本所花费的时间是1.67 mins
处理完前70000个样本所花费的时间是1.67 mins
处理完前80000个样本所花费的时间是1.67 mins
处理完前90000个样本所花费的时间是1.67 mins
处理完前100000个样本所花费的时间是1.67 mins
处理完前110000个样本所花费的时间是1.67 mins
处理完前10000个样本所花费的时间是2.02 mins
处理完前10000个样本所花费的时间是2.30 mins
处理完前20000个样本所花费的时间是2.30 mins
处理完前30000个样本所花费的时间是2.30 mins
处理完前40000个样本所花费的时间是2.30 mins
处理完前50000个样本所花费的时间是2.30 mins
处理完前60000个样本所花费的时间是2.30 mins
处理完前70000个样本所花费的时间是2.30 mins
处理完前10000个样本所花费的时间是2.65 mins
处理完前10000个样本所花费的时间是3.00 mins
处理完前20000个

### 1.3 保持分词后的结果

In [47]:
### 将cutWords 写入local文件##
with open("./NLPData/cutWords_lists1.txt",'w',encoding = 'utf-8') as file:
    for cutword in cutWords:
        file.write(''.join(cutword[0])+'\n')#cutword[0] ->content
file.close()

In [48]:
### 获得文本的对应标签写入单独的文件#
with open("./NLPData/cutWords_labels1.txt",'w',encoding = 'utf-8') as file:
    for cutword in cutWords:
        file.write(''.join(cutword[1])+'\n')#cutword[1] ->label
file.close()

### 1.4重新加载文本序列

In [1]:
cutWords_list = []
with open("./NLPData/cutWords_lists1.txt",'r',encoding = 'utf-8') as file:
    cutWords_list = [k.split() for k in file]

### 2 Word2vec模型训练过程

In [2]:
import time
import warnings 
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec
start = time.time()
word2vec_model  = Word2Vec(cutWords_list,size = 100,window = 2,iter = 100,min_count = 20)
end = time.time()
print("训练%d个样本花费的时间是 %0.2f"%(len(cutWords_list),(end - start)/60),'mins')

训练80000个样本花费的时间是 2.62 mins


In [3]:
## 简单相似度计算 ### 
word2vec_model.wv.most_similar("汽车",topn = 4)

[('电动汽车', 0.6746231913566589),
 ('电动车', 0.6409035921096802),
 ('汽车产业', 0.6318580508232117),
 ('车', 0.6138498783111572)]

In [4]:
## save model ###
word2vec_model.save("./NLPData/word2vec_text_classifier_model.w2v")

In [7]:
### 重新加载模型
from gensim.models import Word2Vec
word2vec_models = Word2Vec.load("./NLPData/word2vec_text_classifier_model.w2v")
word2vec_models.wv.most_similar('技术',topn = 5)

[('关键技术', 0.6680781841278076),
 ('技术创新', 0.6376924514770508),
 ('核心技术', 0.6248726844787598),
 ('人工智能', 0.5972033739089966),
 ('科技', 0.5760294198989868)]

### 3 特征工程 - 找到每个词对应的词向量

In [6]:
### 对于每一篇文章，获取文章的每个分词在word2vec模型中的向量；
### 然后每一篇文章中词分词都可以在word2vec模型中的
### 相关性向量的求和后的平均数，即此篇文章在word2vec模型中的相关向量
def getVector(cutWords,word2vec_models):
    count = 0
    article_vector = np.zeros(word2vec_models.layer1_size)
    for cutWord in cutWords:
        if cutWord in word2vec_models:
            article_vector+= word2vec_models[cutWord]
            count+=1 
    return article_vector/count # 返回的是文档中每个词对应词向量的平均值

In [ ]:
# 取一部分数量的词来更换成词向量
cutword_nums = 50000
start        = time.time()
vector_list  = []
i = 0
for cutWords in cutWords_list[:cutword_nums]:
    vector_list.append(getVector(cutWords,word2vec_models))
    i+=1
    if i%5000 == 0:
        print("前%d篇文章所形成词向量花费的时间%0.2f" %(i,time.time() - start),'sec')
X = np.array(vector_list)
print("处理全部花费的时间:%0.2f"%(time.time() - start),'sec')

In [ ]:
X[0],len(X[0])

In [ ]:
### j将计算的特征矩阵save成文档来
X.dump('./NLPData/articles_vector.txt') # dump方法传入一个字符串

In [ ]:
import numpy as np
X = np.load('./NLPData/articles_vector.txt',allow_pickle = True)

### 4. 数据分割

In [16]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
labelencoder = LabelEncoder()
with open("./NLPData/cutWords_labels.txt",'r',encoding = 'utf-8') as file:
    y_labels = [k.strip() for k in file]
y =  labelencoder.fit_transform(y_labels) # 所有的label y

### 5 模型训练

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 200)
log_clf = LogisticRegression()
log_clf.fit(X_train,y_train)
print("逻辑回归模型准确率:%0.2f"%(log_clf.score(X_test,y_test))*100,"%")

### 通过joblib来保存model
from sklearn.externals import joblib
joblib.dump(logistic_model,'./NLPData/logistic.model')

In [ ]:
### 加载model
from sklearn.externals import joblib
logistic_model = joblib.load('./NLPData/logistic.model')

In [115]:
### 读取特定path下文件夹里面的特定格式的文件，并写入特定文件内###
import os 
import csv 
import pandas as pd
def read_file(path,file,special_format):
    writer = open(path,'w',encoding = 'utf-8')
    for root,dirs,files in os.walk():
        for file in files:
            if os.path.splitext(file)[1] == special_format:
                with open(os.path.join(root,file),'r',encoding = "utf-8") as f:
                    for line in f.readlines():
                        writer.write(line.strip('\n')+'\n')
    writer.close()
#Example:
#file = "./datas/mt/酒店评论数据备份v2/数据分割处理/postive/content/contentMergedPostive.txt"
#path = "./datas/mt/酒店评论数据备份v2/数据分割处理/postive/content"
#special_format = ".txt"
#read_file(path,file,special_format)